# Discharge Notes and Readmission Rates: EDA

This notebook is for the initial exploratory data analysis of the dataset.

## Overview

## Business Understanding

## Data Understanding

## Import

Import all the relevant libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re

In [2]:
#set the pandas display column width as max to read in all the text
#pd.set_option('display.max_colwidth', None)

## Functions

This section is for any functions created for the analysis.

In [3]:
def group_categories(df, category):
    '''
    some of the datasets have multiple rows for diagnosis or ICD code per subject and admission ID
    in order to onehotencode it later, this function aggregates all the codes into a list, and then
    convert the list into a string separate by a comma
    
    the function takes in the dataframe which should only contain three columns: subject_id, hadm_id,
    and the category of interest; and the string for the column name of the cateogry of interest
    '''
    #group by the subject and admission ID
    group = df.groupby(['SUBJECT_ID', 'HADM_ID'])
    #create a list of all the values associated with the subject and admission ID
    result = group.agg({category:lambda x:list(set(x.tolist()))})
    result.reset_index(inplace=True)
    #convert the list into a string, separated by commas
    result[category] = [', '.join(map(str, l)) for l in result[category]]
    return result

## Exploratory Data Analysis (EDA)

In this section, I focused on reviewing the datasets I am using for this project.

### Admission Data

The ADMISSIONS dataset has the information on the demographics of the patient. 

In [4]:
#read in the dataset
admit_df = pd.read_csv('data/ADMISSIONS.csv.gz', compression='gzip')

In [5]:
#review the columns in the dataset
admit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                58976 non-null  int64 
 1   SUBJECT_ID            58976 non-null  int64 
 2   HADM_ID               58976 non-null  int64 
 3   ADMITTIME             58976 non-null  object
 4   DISCHTIME             58976 non-null  object
 5   DEATHTIME             5854 non-null   object
 6   ADMISSION_TYPE        58976 non-null  object
 7   ADMISSION_LOCATION    58976 non-null  object
 8   DISCHARGE_LOCATION    58976 non-null  object
 9   INSURANCE             58976 non-null  object
 10  LANGUAGE              33644 non-null  object
 11  RELIGION              58518 non-null  object
 12  MARITAL_STATUS        48848 non-null  object
 13  ETHNICITY             58976 non-null  object
 14  EDREGTIME             30877 non-null  object
 15  EDOUTTIME             30877 non-null

In [6]:
#review the first 5 entires in the dataset
admit_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [7]:
#check what admission types are included
admit_df['ADMISSION_TYPE'].value_counts()

EMERGENCY    42071
NEWBORN       7863
ELECTIVE      7706
URGENT        1336
Name: ADMISSION_TYPE, dtype: int64

In [8]:
#drop any irrelevant columns
columns_to_drop = ['DEATHTIME','EDREGTIME','EDOUTTIME','HOSPITAL_EXPIRE_FLAG']
admit_df.drop(columns_to_drop, axis=1, inplace=True)

### Chart Notes

The NOTEEVENTS dataset has the chart notes, including discharge notes, per patient and admissions.

In [ ]:
#read in the dataset
notes_df = pd.read_csv('data/NOTEEVENTS.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
notes_df.info()

In [ ]:
#review the first 5 entires in the dataset
notes_df.head()

In [ ]:
#the column ISERROR, if the value is 1, indicates that a physician indicated that the notes are wrong
#drop any columns that do not have an error
notes_df = notes_df[notes_df['ISERROR'] != 1]
#isolate the discharge notes
notes_df = notes_df[notes_df['CATEGORY'] == 'Discharge summary']

In [ ]:
#drop any irrelevant columns
columns_to_drop = ['ROW_ID','CHARTDATE','CHARTTIME','STORETIME','CATEGORY','DESCRIPTION','CGID','ISERROR']
notes_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
#review the edited dataframe
notes_df.head()

In [ ]:
notes_df.info()

In [ ]:
#it seems like there are multiple discharge notes per subject and admission IDs
notes_df.duplicated(subset=['SUBJECT_ID','HADM_ID'], keep=False)

In [ ]:
#combine any texts that have the same SUBJECT_ID and HADM_ID
notes_df = group_categories(notes_df, 'TEXT')

In [ ]:
notes_df

### Additional Patient Data

The PATIENT dataset contains further information of the patient include date of birth and gender

In [ ]:
#read in the dataset
patients_df = pd.read_csv('data/PATIENTS.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
patients_df.info()

In [ ]:
#review the first 5 entires in the dataset
patients_df.head()

In [ ]:
#isolate the relevant columns
patients_df = patients_df[['SUBJECT_ID', 'GENDER', 'DOB']]

### CPT Codes

The CPTEVENTS dataset contains the CPT (Current Procedural Terminology ) codes are five digit number assigned to every service a medical practitioner provided to a patient.

In [ ]:
#read in the dataset
cpt_events_df = pd.read_csv('data/CPTEVENTS.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
cpt_events_df.info()

In [ ]:
#review the first 5 entires in the dataset
cpt_events_df.head()

In [ ]:
#isolate the relevant columns
cpt_events_df = cpt_events_df[['SUBJECT_ID', 'HADM_ID', 'CPT_CD']]

### Diagnosis ICD Codes

The DIAGNOSES dataset contains all diagnoses per subject and admission IDs. The column SEQ_NUM contains the order in which the ICD diagnoses relate to the patient.

In [ ]:
#read in the dataset
diagnoses_icd_df = pd.read_csv('data/DIAGNOSES_ICD.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
diagnoses_icd_df.info()

In [ ]:
#review the first 5 entires in the dataset
diagnoses_icd_df.head()

In [ ]:
#isolate the relevant columns
diagnoses_icd_df = diagnoses_icd_df[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]

### DRG Codes

The DRGCODES contains the DRG (diagnosis-related group) codes, which is a patient classification system that standardizes prospective payment.

In [ ]:
#read in the dataset
drg_codes_df = pd.read_csv('data/DRGCODES.csv.gz', compression='gzip')

In [ ]:
#review the columns in the dataset
drg_codes_df.info()

In [ ]:
#review the first 5 entires in the dataset
drg_codes_df.head()

In [ ]:
#isolate the relevant columns
drg_codes_df = drg_codes_df[['SUBJECT_ID','HADM_ID','DRG_CODE']]

### Procedure ICD Codes

The PROCEDURES_ICD dataset contains the procedure ICD codes per subject and patient IDs.

In [35]:
#read in the dataset
proced_icd_df = pd.read_csv('data/PROCEDURES_ICD.csv.gz', compression='gzip')

In [36]:
#review the columns in the dataset
proced_icd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240095 entries, 0 to 240094
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   ROW_ID      240095 non-null  int64
 1   SUBJECT_ID  240095 non-null  int64
 2   HADM_ID     240095 non-null  int64
 3   SEQ_NUM     240095 non-null  int64
 4   ICD9_CODE   240095 non-null  int64
dtypes: int64(5)
memory usage: 9.2 MB


In [37]:
#review the first 5 entires in the dataset
proced_icd_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [38]:
#isolate the relevant columns
proced_icd_df = proced_icd_df[['SUBJECT_ID','HADM_ID','ICD9_CODE']]

## Searching for 30-Day Readmissions that Were Emergencies

In [39]:
#sort the dates in ascending order, so that if we were to delete the last duplicate, it would be the latest admissions
admit_df.sort_values('ADMITTIME', inplace=True)
admit_df.reset_index(drop=True, inplace=True)

In [40]:
admit_df['ADMITTIME'] = admit_df['ADMITTIME'].str[:10]
admit_df['ADMITTIME'] = pd.to_datetime(admit_df['ADMITTIME'], format="%Y/%m/%d")
admit_df['DISCHTIME'] = admit_df['DISCHTIME'].str[:10]
admit_df['DISCHTIME'] = pd.to_datetime(admit_df['DISCHTIME'], format="%Y/%m/%d")

In [41]:
admit_df['READMITTIME'] = admit_df.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
admit_df['READMIT_TYPE'] = admit_df.groupby('SUBJECT_ID')['ADMISSION_TYPE'].shift(-1)

In [42]:
admit_df[admit_df['READMIT_TYPE'] == 'ELECTIVE']

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE
7,11401,9319,137275,2100-07-01,2100-07-15,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,PROTESTANT QUAKER,WIDOWED,WHITE,LEFT RENAL CA/SDA,1,2102-03-13,ELECTIVE
109,52466,79576,195819,2100-09-18,2100-09-20,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,PROTESTANT QUAKER,MARRIED,WHITE,DYSPNEA;S/P EGD,1,2100-11-05,ELECTIVE
165,16723,13636,155423,2100-10-30,2100-11-03,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,JEWISH,WIDOWED,WHITE,GASTROINTESTINAL BLEED,1,2105-07-01,ELECTIVE
167,43522,51754,105236,2100-10-31,2100-11-03,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,LOWER GI BLEED,1,2100-12-22,ELECTIVE
309,33507,27414,111607,2101-02-14,2101-02-15,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,UNOBTAINABLE,MARRIED,WHITE - OTHER EUROPEAN,"POPLITEAL ANEURYSM, RIGHT/SDA",0,2101-02-28,ELECTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58668,50668,73946,100104,2201-06-21,2201-07-03,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Government,ENGL,NOT SPECIFIED,SINGLE,OTHER,PERFORATED BOWEL,1,2201-10-22,ELECTIVE
58698,22286,18250,132130,2201-07-29,2201-09-04,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,ENGL,PROTESTANT QUAKER,SINGLE,BLACK/AFRICAN AMERICAN,CENTRAL NERVOUS SYSTEM LYMPHOMA,1,2201-09-09,ELECTIVE
58764,12401,10144,172618,2202-06-04,2202-06-05,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,ESOPHAGEAL FOREIGN BODY,1,2202-10-11,ELECTIVE
58820,5933,4900,117413,2203-06-29,2203-07-08,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Medicare,ENGL,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,RECTAL CANCER/SDA,1,2203-10-26,ELECTIVE


In [43]:
elective_index = admit_df['READMIT_TYPE'] == 'ELECTIVE'
admit_df.loc[elective_index,'READMITTIME'] = pd.NaT
admit_df.loc[elective_index,'READMIT_TYPE'] = np.NaN

In [44]:
admit_df['READMITTIME'] = admit_df.groupby(['SUBJECT_ID'])['READMITTIME'].fillna(method = 'bfill')
admit_df['READMIT_TYPE'] = admit_df.groupby(['SUBJECT_ID'])['READMIT_TYPE'].fillna(method = 'bfill')

In [45]:
#convert the discharge time and the date of birth to time series
ts_disch = admit_df['DISCHTIME'].dt.to_pydatetime()
ts_readmit = admit_df['READMITTIME'].dt.to_pydatetime()

In [46]:
#subtract the dischrage time by the date of birth, and divide by 365 to get years
readmit_df = pd.DataFrame((ts_readmit-ts_disch))

In [47]:
readmit_df.head(10)

,0
0,NaT
1,NaT
2,NaT
3,NaT
4,NaT
5,254 days
6,590 days
7,NaT
8,91 days
9,NaT


In [48]:
readmit_df[0].value_counts()

1 days       245
2 days       244
3 days       211
5 days       176
4 days       171
            ... 
3637 days      1
2212 days      1
1693 days      1
1836 days      1
445 days       1
Name: 0, Length: 1989, dtype: int64

In [49]:
#retrieve the age, but note that because the structure of the datetime data, dt.days will return the int value of whatever is in the columns. Since I
#already divided by 365, when I use dt.days, I am actually getting the years
admit_df['DAYS_READMIT'] = readmit_df[0].dt.days

In [50]:
admit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                58976 non-null  int64         
 1   SUBJECT_ID            58976 non-null  int64         
 2   HADM_ID               58976 non-null  int64         
 3   ADMITTIME             58976 non-null  datetime64[ns]
 4   DISCHTIME             58976 non-null  datetime64[ns]
 5   ADMISSION_TYPE        58976 non-null  object        
 6   ADMISSION_LOCATION    58976 non-null  object        
 7   DISCHARGE_LOCATION    58976 non-null  object        
 8   INSURANCE             58976 non-null  object        
 9   LANGUAGE              33644 non-null  object        
 10  RELIGION              58518 non-null  object        
 11  MARITAL_STATUS        48848 non-null  object        
 12  ETHNICITY             58976 non-null  object        
 13  DIAGNOSIS       

In [51]:
admit_df['DAYS_READMIT'].fillna(1000,inplace=True)

In [52]:
admit_df['READMISSION'] = 0

In [53]:
admit_df.loc[(admit_df['DAYS_READMIT'] > 0) & (admit_df['DAYS_READMIT'] < 31), 'READMISSION'] = 1

In [54]:
admit_df['READMISSION'].value_counts()

0    55782
1     3194
Name: READMISSION, dtype: int64

Since some patients are readmitted multiple times at the hospital, I wanted to remove the last visit they had to the hospital since I cannot use the discharge note for the last visit since there would be no subsequent readmissions

In [55]:
#sort the dates in ascending order, so that if we were to delete the last duplicate, it would be the latest admissions
admit_df.sort_values('ADMITTIME', inplace=True)

In [56]:
#create a dataframe on the number of times a patient was admitted
dupli_admit_df = pd.DataFrame(admit_df.groupby('SUBJECT_ID')['HADM_ID'].count())
dupli_admit_df.rename(columns={'HADM_ID': 'ADMIT_COUNTS'}, inplace=True)

In [57]:
#print out how many patients were admitted once vs admitted multiple times
print(dupli_admit_df[dupli_admit_df['ADMIT_COUNTS']==1].count())
print(dupli_admit_df[dupli_admit_df['ADMIT_COUNTS']>1].count())

ADMIT_COUNTS    38983
dtype: int64
ADMIT_COUNTS    7537
dtype: int64


We can see here that there is clearly a class imbalance that we will have to take into account when modeling.

In [58]:
#merged the admission counts back into the main dataframe
new_df = admit_df.merge(dupli_admit_df,on='SUBJECT_ID')

#separate out the people who were never readmitted
mod_admit_df = new_df[new_df['ADMIT_COUNTS']==1]
mod_admit_df['ADMISSION_TYPE'].value_counts()

EMERGENCY    25417
NEWBORN       7603
ELECTIVE      5066
URGENT         897
Name: ADMISSION_TYPE, dtype: int64

In [59]:
#separate out the people who were readmitted multiple times
duplicate_df = new_df[new_df['ADMIT_COUNTS']>1]

#create a copy of the dataframe with the duplicates
df1 = duplicate_df.copy()

In [60]:
#create a list of the unique number of times people were readmitted
admit_counts = sorted(df1['ADMIT_COUNTS'].unique())

In [61]:
#for every admission counts
for i in admit_counts:
    #find the patients associated with that admission count
    df2 = df1[df1['ADMIT_COUNTS']==i]
    
    #this while loop is to iterate the dataframe and retrieving the first duplicate of the main dataframe
    #and then the main dataframe becomes the deleted out duplicates, this process is repeated until the 
    #admin counts reaches 1 at 'i -=1'
    while i > 1:
        #this dataframe holds the duplicates
        df3 = df2.loc[df2.duplicated(subset='SUBJECT_ID')]
        #this datafame holds the de-duped dataframe
        df4 = df2.drop_duplicates('SUBJECT_ID')
        #add the deduped dataframe to the main dataframe
        mod_admit_df = pd.concat([mod_admit_df, df4])
        df2 = df3
        i -= 1

In [62]:
#review the modified DataFrame to see if it worked
mod_admit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51439 entries, 0 to 47700
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                51439 non-null  int64         
 1   SUBJECT_ID            51439 non-null  int64         
 2   HADM_ID               51439 non-null  int64         
 3   ADMITTIME             51439 non-null  datetime64[ns]
 4   DISCHTIME             51439 non-null  datetime64[ns]
 5   ADMISSION_TYPE        51439 non-null  object        
 6   ADMISSION_LOCATION    51439 non-null  object        
 7   DISCHARGE_LOCATION    51439 non-null  object        
 8   INSURANCE             51439 non-null  object        
 9   LANGUAGE              28220 non-null  object        
 10  RELIGION              50994 non-null  object        
 11  MARITAL_STATUS        41633 non-null  object        
 12  ETHNICITY             51439 non-null  object        
 13  DIAGNOSIS       

In [63]:
mod_admit_df[mod_admit_df['ADMIT_COUNTS']==3].sort_values('SUBJECT_ID').head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION,ADMIT_COUNTS
18056,37,36,122659,2131-05-12,2131-05-25,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHEST PAIN/SHORTNESS OF BREATH,1,NaT,NaN,1000.0,0,3
18055,36,36,182104,2131-04-30,2131-05-08,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,2131-05-12,EMERGENCY,4.0,1,3
8491,112,107,191941,2115-02-20,2115-02-21,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,AV FISTULA;ANGIOPLASTY,1,2121-11-30,EMERGENCY,2474.0,0,3
8492,113,107,182383,2121-11-30,2121-12-05,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,SEPARATED,HISPANIC OR LATINO,HYPERKALEMIA;CONGESTIVE HEART FAILURE,1,2122-05-14,EMERGENCY,160.0,0,3
18407,186,138,188284,2133-12-21,2134-01-11,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,HEMATURIA;RENAL MASS,1,NaT,NaN,1000.0,0,3
18406,185,138,108120,2131-10-31,2131-11-06,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\ AORTIC VALVE REPLACEMENT/SDA,1,2133-12-21,EMERGENCY,776.0,0,3
10164,205,154,162891,2118-04-05,2118-04-11,URGENT,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,EPISCOPALIAN,MARRIED,WHITE,UNSTABLE ANGINA\CATH WITH BRACHY S/B,1,2127-12-23,EMERGENCY,3543.0,0,3
10163,204,154,111735,2117-12-29,2118-01-01,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,EPISCOPALIAN,MARRIED,WHITE,ACUTE MYOCARDIAL INFARCTION,1,2118-04-05,URGENT,94.0,0,3
20288,308,236,182562,2135-05-26,2135-05-28,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,PATIENT DECLINED TO ANSWER,POST OP BLEEDING,1,2139-02-18,EMERGENCY,1362.0,0,3
20287,307,236,151459,2134-10-04,2134-10-15,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,PATIENT DECLINED TO ANSWER,END STAGE LIVER DISEASE,1,2135-05-26,EMERGENCY,223.0,0,3


As we can see the patients who were admitted three times showed up twice in our dataframe, meaning that one duplicate has successfully been removed. Below, we pull subject ID 36 as an example from the original dataframe to see that the last admission visit was removed.

In [64]:
#double checking that the last admissions time was the one that was removed
admit_df[admit_df['SUBJECT_ID'] == 36]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION
17808,36,36,182104,2131-04-30,2131-05-08,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,2131-05-12,EMERGENCY,4.0,1
17824,37,36,122659,2131-05-12,2131-05-25,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHEST PAIN/SHORTNESS OF BREATH,1,NaT,NaN,1000.0,0
19747,38,36,165660,2134-05-10,2134-05-20,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,VENTRAL HERNIA/SDA,1,NaT,NaN,1000.0,0


In [65]:
#convert all the admission counts to 1 (readmission occurred after this admissions) or 0 (no readmission)
mod_admit_df.loc[mod_admit_df['ADMIT_COUNTS'] == 1, 'ADMIT_COUNTS'] = 0
mod_admit_df.loc[mod_admit_df['ADMIT_COUNTS'] > 1, 'ADMIT_COUNTS'] = 1
mod_admit_df.rename(columns={'ADMIT_COUNTS': 'READMISSION'}, inplace=True)

## Grouping Codes by Subject and Admission IDs

Some of the datasets need to be grouped by subject and admission IDs such as the diagonsis and ICD codes, since each admissions could have multiple codes.

In [66]:
#group the CPT dataset
cpt_events_df = group_categories(cpt_events_df, 'CPT_CD')

In [67]:
#group the diagnoses dataset
diagnoses_icd_df = group_categories(diagnoses_icd_df, 'ICD9_CODE')
diagnoses_icd_df.rename(columns={'ICD9_CODE': 'DIAG_ICD9_CODE'}, inplace=True)

In [68]:
#group the DRG dataset
drg_codes_df = group_categories(drg_codes_df, 'DRG_CODE')

In [69]:
#group the procedures dataset
proced_icd_df = group_categories(proced_icd_df, 'ICD9_CODE')
proced_icd_df.rename(columns={'ICD9_CODE': 'PROCED_ICD9_CODE'}, inplace=True)

## Merging Dataframes

Here, I merged some of the datasets based on the subject and admissions IDs.

The modified admissions and PATIENTS datasets can be merged using the subject ID. The PATIENTS dataset has the date of birth (DOB) of the patient, which can be subtracted by the discharge date to get the age of the patient. However, patients who are over 89 years old had their DOB shifted to 300 years after the admission date to comply with HIPAA regulations.

In [70]:
#merge the modified admissions dataset with the PATIENTS dataset
mod_admit_df = admit_df.merge(patients_df, on='SUBJECT_ID')

In [71]:
#obtain the year, month, and date of the discharge time
mod_admit_df['DOB'] = mod_admit_df['DOB'].str[:10]

In [72]:
#review the modified discharge time
mod_admit_df['DISCHTIME']

0       2100-06-09
1       2100-06-19
2       2100-06-27
3       2100-06-17
4       2100-06-30
           ...    
58971   2201-08-04
58972   2201-08-12
58973   2201-11-06
58974   2201-11-27
58975   2202-03-18
Name: DISCHTIME, Length: 58976, dtype: datetime64[ns]

In [73]:
#convert the discharge time and the date of birth into datetime
mod_admit_df['DOB'] = pd.to_datetime(mod_admit_df['DOB'], format="%Y/%m/%d")

In [74]:
#convert the discharge time and the date of birth to time series
ts_dob = mod_admit_df['DOB'].dt.to_pydatetime()

In [75]:
#subtract the dischrage time by the date of birth, and divide by 365 to get years
age_df = pd.DataFrame((ts_disch-ts_dob)//365)

In [76]:
#retrieve the age, but note that because the structure of the datetime data, dt.days will return the int value of whatever is in the columns. Since I
#already divided by 365, when I use dt.days, I am actually getting the years
mod_admit_df['AGE'] = age_df[0].dt.days

In [77]:
#check the value counts of the age to see how many of the values are 300
mod_admit_df['AGE'].value_counts()

0      8073
300    2607
78     1140
77     1123
69     1103
       ... 
301       1
91        1
305       1
306       1
7         1
Name: AGE, Length: 92, dtype: int64

About 2K admissions were of patients who were older than 89. In order to adjust the number, I assumed that all people ages "300" are 90.

In [78]:
#change anyone with an age of 300 to 90
mod_admit_df.loc[mod_admit_df['AGE'] == 300, "AGE"] = 90

In [79]:
mod_admit_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION,GENDER,DOB,AGE
0,53486,82574,118464,2100-06-07,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,...,OTHER,HYPOTENSION,1,NaT,NaN,1000.0,0,M,2044-04-23,56
1,35574,29156,161773,2100-06-09,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,...,WHITE,RESPIRATORY DISTRESS,1,NaT,NaN,1000.0,0,M,2028-04-23,72
2,14753,12001,173927,2100-06-14,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,WHITE,RT BASAL GANGLIA BLEEDING,1,NaT,NaN,1000.0,0,F,2028-10-27,71
3,25723,21081,159656,2100-06-14,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,...,BLACK/AFRICAN AMERICAN,SHORTNESS OF BREATH,1,NaT,NaN,1000.0,0,F,2067-04-01,33
4,38932,32096,158366,2100-06-22,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,...,WHITE,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",1,NaT,NaN,1000.0,0,F,2070-12-13,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,10977,8982,192942,2201-07-24,2201-08-04,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,NaN,...,WHITE,NEWBORN,1,NaT,NaN,1000.0,0,M,2201-07-24,7
58972,9040,7391,153695,2201-07-26,2201-08-12,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,WHITE,CROHN'S DISEASE; ENTEROCUTANEOUS FISTULA,1,2201-08-26,EMERGENCY,14.0,1,F,2133-07-08,75
58973,9041,7391,135421,2201-08-26,2201-11-06,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,NaN,...,WHITE,ABDOMINAL ABCESS,1,2201-11-17,EMERGENCY,11.0,1,F,2133-07-08,76
58974,9042,7391,112425,2201-11-17,2201-11-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,WHITE,SEPSIS,1,2201-12-08,EMERGENCY,11.0,1,F,2133-07-08,76


In addition, I merged the CPT, 

In [80]:
#merge the admissions dataframe
main_df = mod_admit_df.merge(notes_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
main_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION,GENDER,DOB,AGE,TEXT
0,53486,82574,118464,2100-06-07,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,...,HYPOTENSION,1,NaT,NaN,1000.0,0,M,2044-04-23,56,Admission Date: [**2100-6-7**] D...
1,35574,29156,161773,2100-06-09,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,...,RESPIRATORY DISTRESS,1,NaT,NaN,1000.0,0,M,2028-04-23,72,Admission Date: [**2100-6-9**] D...
2,14753,12001,173927,2100-06-14,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,RT BASAL GANGLIA BLEEDING,1,NaT,NaN,1000.0,0,F,2028-10-27,71,Admission Date: [**2100-6-14**] Dischar...
3,25723,21081,159656,2100-06-14,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,...,SHORTNESS OF BREATH,1,NaT,NaN,1000.0,0,F,2067-04-01,33,Admission Date: [**2100-6-14**] Discharge...
4,38932,32096,158366,2100-06-22,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,...,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",1,NaT,NaN,1000.0,0,F,2070-12-13,29,Admission Date: [**2100-6-22**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52721,10977,8982,192942,2201-07-24,2201-08-04,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,M,2201-07-24,7,Admission Date: [**2201-7-24**] Dischar...
52722,9040,7391,153695,2201-07-26,2201-08-12,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,CROHN'S DISEASE; ENTEROCUTANEOUS FISTULA,1,2201-08-26,EMERGENCY,14.0,1,F,2133-07-08,75,Admission Date: [**2201-7-26**] ...
52723,9041,7391,135421,2201-08-26,2201-11-06,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,NaN,...,ABDOMINAL ABCESS,1,2201-11-17,EMERGENCY,11.0,1,F,2133-07-08,76,Admission Date: [**2201-8-26**] ...
52724,9042,7391,112425,2201-11-17,2201-11-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,SEPSIS,1,2201-12-08,EMERGENCY,11.0,1,F,2133-07-08,76,Admission Date: [**2201-11-17**] ...


In [81]:
print("The modified admissions dataset has", str(len(mod_admit_df)),"rows.")
print("The notes dataset has", str(len(notes_df)),"rows.")
print("The merged dataset has", str(len(main_df)),"rows.")

The modified admissions dataset has 58976 rows.
The notes dataset has 52726 rows.
The merged dataset has 52726 rows.


From this inner merge, about 6K rows were lost. Below I explore where there are null values after doing an 'outer' merge between the admissions and notes datasets. 

In [82]:
#performing outer merge to see where the null values are
main2_df = mod_admit_df.merge(notes_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
main2_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION,GENDER,DOB,AGE,TEXT
0,53486,82574,118464,2100-06-07,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,...,HYPOTENSION,1,NaT,NaN,1000.0,0,M,2044-04-23,56,Admission Date: [**2100-6-7**] D...
1,35574,29156,161773,2100-06-09,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,...,RESPIRATORY DISTRESS,1,NaT,NaN,1000.0,0,M,2028-04-23,72,Admission Date: [**2100-6-9**] D...
2,14753,12001,173927,2100-06-14,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,RT BASAL GANGLIA BLEEDING,1,NaT,NaN,1000.0,0,F,2028-10-27,71,Admission Date: [**2100-6-14**] Dischar...
3,25723,21081,159656,2100-06-14,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,...,SHORTNESS OF BREATH,1,NaT,NaN,1000.0,0,F,2067-04-01,33,Admission Date: [**2100-6-14**] Discharge...
4,38932,32096,158366,2100-06-22,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,...,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",1,NaT,NaN,1000.0,0,F,2070-12-13,29,Admission Date: [**2100-6-22**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,10977,8982,192942,2201-07-24,2201-08-04,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,M,2201-07-24,7,Admission Date: [**2201-7-24**] Dischar...
58972,9040,7391,153695,2201-07-26,2201-08-12,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,CROHN'S DISEASE; ENTEROCUTANEOUS FISTULA,1,2201-08-26,EMERGENCY,14.0,1,F,2133-07-08,75,Admission Date: [**2201-7-26**] ...
58973,9041,7391,135421,2201-08-26,2201-11-06,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,NaN,...,ABDOMINAL ABCESS,1,2201-11-17,EMERGENCY,11.0,1,F,2133-07-08,76,Admission Date: [**2201-8-26**] ...
58974,9042,7391,112425,2201-11-17,2201-11-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,SEPSIS,1,2201-12-08,EMERGENCY,11.0,1,F,2133-07-08,76,Admission Date: [**2201-11-17**] ...


In [83]:
#reviewing where there is a null for the text value in the merged file from the notes dataset
null_text_df = main2_df[main2_df['TEXT'].isna()]
null_text_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION,GENDER,DOB,AGE,TEXT
17,35034,28711,104132,2100-07-02,2100-07-07,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,M,2100-07-02,0,NaN
21,602,476,161042,2100-07-05,2100-07-07,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,F,2100-07-05,0,NaN
45,43523,51754,182977,2100-12-22,2100-12-24,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,...,OBSTRUCTIVE HYPERTROPHIC CARDIOMYOPATHY\ETHANO...,1,NaT,NaN,1000.0,0,M,2055-07-09,45,NaN
46,33653,27526,134575,2100-07-29,2100-08-03,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,F,2100-07-29,0,NaN
54,2959,2437,104692,2100-08-03,2100-08-05,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,M,2100-08-03,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58941,22936,18806,101513,2201-06-10,2201-06-14,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,F,2201-06-10,4,NaN
58957,13969,11400,109331,2201-06-25,2201-06-27,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,M,2201-06-25,5,NaN
58961,29635,24254,118562,2201-06-29,2201-07-03,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,...,NEWBORN,1,NaT,NaN,1000.0,0,M,2201-06-29,5,NaN
58967,51990,77930,151564,2201-07-14,2201-07-21,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,ENGL,...,AFIB WITH RVR,1,NaT,NaN,1000.0,0,F,1901-07-14,306,NaN


It seems like most of the rows that were lost during the merge were because of null discharge notes.

In [84]:
null_text_df['ADMISSION_TYPE'].value_counts()

NEWBORN      4220
EMERGENCY    1598
ELECTIVE      375
URGENT         57
Name: ADMISSION_TYPE, dtype: int64

Based on the admission types, ~4.2K of the rows were newborns and therefore they may not need a discharge summary. I will remove them to explore why the remaining ~1.6K were dropped.

In [85]:
null_text_df = null_text_df[null_text_df['ADMISSION_TYPE'] != 'NEWBORN']
null_text_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMITTIME,READMIT_TYPE,DAYS_READMIT,READMISSION,GENDER,DOB,AGE,TEXT
45,43523,51754,182977,2100-12-22,2100-12-24,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,...,OBSTRUCTIVE HYPERTROPHIC CARDIOMYOPATHY\ETHANO...,1,NaT,NaN,1000.0,0,M,2055-07-09,45,NaN
113,44027,53252,189277,2100-09-13,2100-09-13,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,TAMPONADE,1,NaT,NaN,1000.0,0,M,1800-09-13,90,NaN
155,25392,20801,138996,2100-10-09,2100-10-15,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,...,HYPONATREMIA,1,2102-03-10,EMERGENCY,511.0,0,F,2036-07-29,64,NaN
280,27403,22465,105036,2101-01-02,2101-01-04,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Private,NaN,...,S/P 30 FEET FALL,1,NaT,NaN,1000.0,0,M,2067-09-13,33,NaN
346,33507,27414,111607,2101-02-14,2101-02-15,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,...,"POPLITEAL ANEURYSM, RIGHT/SDA",0,NaT,NaN,1000.0,0,M,2030-11-26,70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58874,16874,13775,196356,2201-04-18,2201-04-18,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Self Pay,NaN,...,SKULL FRACTURE,1,NaT,NaN,1000.0,0,M,2163-04-09,41,NaN
58911,27212,22296,193653,2201-05-22,2201-05-28,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,HAIT,...,INTRACRANIAL HEMORRHAGE,1,NaT,NaN,1000.0,0,M,2123-05-09,81,NaN
58920,4848,3996,164184,2201-05-27,2201-06-07,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Self Pay,NaN,...,LIVER LACERATION;PNEUMOTHORAX,1,NaT,NaN,1000.0,0,M,2177-11-11,27,NaN
58939,50684,74001,123280,2201-06-10,2201-06-12,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,...,PARALYSIS,1,2203-02-26,EMERGENCY,624.0,0,M,2163-02-03,43,NaN


In [86]:
null_text_df['DISCHARGE_LOCATION'].value_counts()

HOME                         1108
DEAD/EXPIRED                  589
HOME HEALTH CARE              148
LEFT AGAINST MEDICAL ADVI      54
SNF                            40
REHAB/DISTINCT PART HOSP       20
DISC-TRAN CANCER/CHLDRN H      19
SHORT TERM HOSPITAL            19
LONG TERM CARE HOSPITAL        12
HOSPICE-HOME                    7
DISCH-TRAN TO PSYCH HOSP        5
OTHER FACILITY                  4
HOSPICE-MEDICAL FACILITY        2
ICF                             2
HOME WITH HOME IV PROVIDR       1
Name: DISCHARGE_LOCATION, dtype: int64

From the discharge location column, about 600 people were either dead/expired or left against medical advice. This further explains some of the null values in the text columns. The remaining ~1K there may be other reasons, but I am assuming that I can drop these rows for now since there are no discharge notes.

I merged the CPT, diagnosis, procedure, DRG codes datasets into one dataframe.

In [87]:
#merge together the four datasets above
merged_df = cpt_events_df.merge(diagnoses_icd_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df = merged_df.merge(drg_codes_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df = merged_df.merge(proced_icd_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df

,SUBJECT_ID,HADM_ID,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,3,145834,"94003, 94002","6826, 0389, 5849, 41071, 4280, 2639, 4275, 425...",416,"9604, 8964, 966, 9672, 9962, 3893"
1,4,185777,"99232, 99233, 99238, 99223, 99231","5715, E9317, 042, 04111, V090, 1363, 2763, 790...",489,"8872, 3323, 3893"
2,6,107064,"99232, 99233, 99239, 99255, 99291, 99231","2766, 9972, 40391, 2767, V1582, 2753, 4440, 2859",302,"5569, 9904, 3957, 91, 3806"
3,9,150750,94003,"431, 5849, 5070, 4280, 2765, 4019",14,"9672, 9604"
4,11,194540,"99233, 99239",1913,"2, 211","9925, 113, 9229, 159"
...,...,...,...,...,...,...
58971,99017,103056,NaN,"3659, 4439, 41401, 41071, 496, 2859","1742, 247","66, 40, 47, 3606, 3607"
58972,99088,165352,NaN,"2326, 43889, 72989, 4019","578, 3611","864, 8661"
58973,99487,188460,NaN,"25000, 2724, 3669, 2859, 41021, 41401, 2689, 5...","1742, 247","66, 40, 3722, 45, 3607, 8856"
58974,99650,190854,NaN,"40391, 25060, 2724, 25050, 3572, 4139, 36201, ...","554, 1733","3929, 3995"


In [88]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 0 to 58975
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   SUBJECT_ID        58976 non-null  int64 
 1   HADM_ID           58976 non-null  int64 
 2   CPT_CD            44148 non-null  object
 3   DIAG_ICD9_CODE    58976 non-null  object
 4   DRG_CODE          58890 non-null  object
 5   PROCED_ICD9_CODE  52243 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.1+ MB


In [89]:
#merge in the previously merged dataframe (cpt, diagnosis, procedure, drg codes) with the main_df
main_df = main_df.merge(merged_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
main_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DAYS_READMIT,READMISSION,GENDER,DOB,AGE,TEXT,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,53486,82574,118464,2100-06-07,2100-06-09,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,...,1000.0,0,M,2044-04-23,56,Admission Date: [**2100-6-7**] D...,"99233, 99291, 99232","2449, 4400, 19889, V4589, 6826, V5869, 4589, 2...","2072, 2073, 314",9229
1,35574,29156,161773,2100-06-09,2100-06-19,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,...,1000.0,0,M,2028-04-23,72,Admission Date: [**2100-6-9**] D...,"99253, 99292, 99233, 99291, 94002, 94003, 99239","4821, 05312, 0417, 2859, 2948, 48241, 2639, 49...","1304, 565","9604, 966, 9672, 3893, 9915"
2,14753,12001,173927,2100-06-14,2100-06-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,1000.0,0,F,2028-10-27,71,Admission Date: [**2100-6-14**] Dischar...,NaN,"7070, 3310, 431, 29410, 725, 2720, 5990, 4019",14,"4513, 966, 4311"
3,25723,21081,159656,2100-06-14,2100-06-17,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,...,1000.0,0,F,2067-04-01,33,Admission Date: [**2100-6-14**] Discharge...,"99233, 99291","49391, 7863, 30470, 042, 96501, 5070, 9708, E8...",87,NaN
4,38932,32096,158366,2100-06-22,2100-06-30,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,...,1000.0,0,F,2070-12-13,29,Admission Date: [**2100-6-22**] ...,"27228, 27252, 27762, 99254","80704, 8240, 8910, 30470, 8080, V4586, 3051, 8...","9123, 486","7906, 7939, 7909, 390, 9904, 8659, 3893"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52721,10977,8982,192942,2201-07-24,2201-08-04,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,NaN,...,1000.0,0,M,2201-07-24,7,Admission Date: [**2201-7-24**] Dischar...,NaN,"76516, 76525, V290, 769, V3001, 7742, 77081",385,"9604, 966, 9671, 9390, 9915, 9983"
52722,9040,7391,153695,2201-07-26,2201-08-12,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,14.0,1,F,2133-07-08,75,Admission Date: [**2201-7-26**] ...,NaN,"4019, 2859, V442, 5680, 56981, V1279, 99889, 9...","148, 2212","4674, 4679, 9904, 4562, 5459, 9915, 4319"
52723,9041,7391,135421,2201-08-26,2201-11-06,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,NaN,...,11.0,1,F,2133-07-08,76,Admission Date: [**2201-8-26**] ...,"99232, 99254","8072, 2761, 99592, 311, 25000, 5961, 8052, 293...","416, 7204","966, 14, 9904, 3893, 4632, 9915, 8604"
52724,9042,7391,112425,2201-11-17,2201-11-27,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,11.0,1,F,2133-07-08,76,Admission Date: [**2201-11-17**] ...,NaN,"25000, 5559, 0389, 99592, 1122, V444, 56981, 7...","576, 7204","9904, 9915, 3893, 966"


No rows were lost from the main_df (admission and notes combined) after merging with the merged_df (CPT, diagnosis, procedure, DRG codes).

In [90]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52726 entries, 0 to 52725
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                52726 non-null  int64         
 1   SUBJECT_ID            52726 non-null  int64         
 2   HADM_ID               52726 non-null  int64         
 3   ADMITTIME             52726 non-null  datetime64[ns]
 4   DISCHTIME             52726 non-null  datetime64[ns]
 5   ADMISSION_TYPE        52726 non-null  object        
 6   ADMISSION_LOCATION    52726 non-null  object        
 7   DISCHARGE_LOCATION    52726 non-null  object        
 8   INSURANCE             52726 non-null  object        
 9   LANGUAGE              32358 non-null  object        
 10  RELIGION              52312 non-null  object        
 11  MARITAL_STATUS        46870 non-null  object        
 12  ETHNICITY             52726 non-null  object        
 13  DIAGNOSIS       

In [91]:
main_df['READMISSION'].value_counts()

0    49715
1     3011
Name: READMISSION, dtype: int64

There seems to be some class imbalance between readmissions and no readmissions, but it does not seem to be very high.

## Dropping Columns and Handling Missing Values

In [92]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52726 entries, 0 to 52725
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                52726 non-null  int64         
 1   SUBJECT_ID            52726 non-null  int64         
 2   HADM_ID               52726 non-null  int64         
 3   ADMITTIME             52726 non-null  datetime64[ns]
 4   DISCHTIME             52726 non-null  datetime64[ns]
 5   ADMISSION_TYPE        52726 non-null  object        
 6   ADMISSION_LOCATION    52726 non-null  object        
 7   DISCHARGE_LOCATION    52726 non-null  object        
 8   INSURANCE             52726 non-null  object        
 9   LANGUAGE              32358 non-null  object        
 10  RELIGION              52312 non-null  object        
 11  MARITAL_STATUS        46870 non-null  object        
 12  ETHNICITY             52726 non-null  object        
 13  DIAGNOSIS       

In [93]:
#drop any irrelevant columns
columns_to_drop = ['ROW_ID','SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','HAS_CHARTEVENTS_DATA',
                   'READMITTIME','READMIT_TYPE','DAYS_READMIT','DOB']
main_df.drop(columns_to_drop, axis=1, inplace=True)

There are missing values in the following columns: LANGUAGE, RELIGION, MARITAL STATUS, DIAGNOSIS, CPT_CD, DRG_CODE, and PROCED_ICD9_CODE. Given that all these columns are categorical columns, I decided to fill all the nulls with "Unknown".

In [94]:
main_df.fillna("UNKNOWN", inplace=True)

In [95]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52726 entries, 0 to 52725
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ADMISSION_TYPE      52726 non-null  object
 1   ADMISSION_LOCATION  52726 non-null  object
 2   DISCHARGE_LOCATION  52726 non-null  object
 3   INSURANCE           52726 non-null  object
 4   LANGUAGE            52726 non-null  object
 5   RELIGION            52726 non-null  object
 6   MARITAL_STATUS      52726 non-null  object
 7   ETHNICITY           52726 non-null  object
 8   DIAGNOSIS           52726 non-null  object
 9   READMISSION         52726 non-null  int64 
 10  GENDER              52726 non-null  object
 11  AGE                 52726 non-null  int64 
 12  TEXT                52726 non-null  object
 13  CPT_CD              52726 non-null  object
 14  DIAG_ICD9_CODE      52726 non-null  object
 15  DRG_CODE            52726 non-null  object
 16  PROCED_ICD9_CODE    52

In [79]:
main_df.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,READMISSION,GENDER,AGE,TEXT,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,CATHOLIC,SINGLE,OTHER,HYPOTENSION,0,M,56,Admission Date: [**2100-6-7**] D...,"99232, 99291, 99233","311, 6826, 7804, 1991, 3051, 4400, 2720, 28529...","2072, 2073, 314",9229
1,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,UNOBTAINABLE,MARRIED,WHITE,RESPIRATORY DISTRESS,0,M,72,Admission Date: [**2100-6-9**] D...,"99291, 99239, 99253, 94002, 99292, 94003, 99233","51881, 48241, 2859, 4821, 5990, 53081, 2639, 0...","1304, 565","9604, 966, 9672, 3893, 9915"
2,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,UNKNOWN,CATHOLIC,MARRIED,WHITE,RT BASAL GANGLIA BLEEDING,0,F,71,Admission Date: [**2100-6-14**] Dischar...,UNKNOWN,"7070, 4019, 29410, 431, 2720, 725, 5990, 3310",14,"4513, 966, 4311"
3,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,UNKNOWN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,SHORTNESS OF BREATH,0,F,33,Admission Date: [**2100-6-14**] Discharge...,"99233, 99291","51881, E8500, 7863, 30470, 042, 5070, 9708, 49...",87,UNKNOWN
4,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,"STERNAL FRACTURE,RIB FRACTURE,LACERATIONS",0,F,29,Admission Date: [**2100-6-22**] ...,"99254, 27252, 27762, 27228","V4586, 27801, 8910, 8074, 8240, 8080, 86121, 3...","9123, 486","7906, 7939, 7909, 390, 9904, 8659, 3893"


## Export Cleaned and Merged Dataset

Export the cleaned and merged dataset into the data folder. I am planning to preprocess the text data in the modeling notebook instead to adjust the preprocessing step to see how it affects the model performance.

In [96]:
#export the main dataframe to the data folder for analysis in the modeling notebook
main_df.to_csv("data/admissions_cleaned_2.csv.gz" , index = False,compression='gzip')